### Bring Your Own Container (BYOC)

In this notebook, we will cover how to bring our own container with either a framework or algorithm to train a model on SageMaker. 

We will use fastai in this case and build our container with custom training code integrated into the container. The other option is to use script mode which is easily done by changing the entrypoint.

Outline:
1. Build your own docker container with fastai framework
2. Register your docker container to Elastic Container Registry (ECR)
3. Use your docker container and prepared data to train your model


#### Container Image
Let's start with building a container image locally and then push that to ECR (Elastic Container Registry)

In [1]:
%cd docker

/home/ec2-user/SageMaker/pssummitwkshp/byoc/docker


In [2]:
!docker build -t fastai .

Sending build context to Docker daemon  6.656kB
Step 1/8 : FROM fastdotai/fastai:latest
latest: Pulling from fastdotai/fastai

55322776: Pulling fs layer 
02cb4e21: Pulling fs layer 
b1d03d39: Pulling fs layer 
e659dcd9: Pulling fs layer 
93dd13fb: Pulling fs layer 
51f98875: Pulling fs layer 
47da6bdc: Pulling fs layer 
e1ed0a6f: Pulling fs layer 
e271bff3: Pulling fs layer 
b700ef54: Pulling fs layer 
3c0e6050: Pulling fs layer 
d68ecf74: Pulling fs layer 
f559d1a8: Pulling fs layer 
6a4fe0a9: Pulling fs layer 
Digest: sha256:a45d4490db48b3eac939d91b46e13c7480bc7e49359cc8875120e9858b7f107bDownloading  424.7MB/719.6MBDownloading  1.045GB/2.962GBDownloading  1.051GB/2.962GBExtracting  254.6MB/2.962GBExtracting  1.186GB/2.962GBExtracting  2.137GB/2.962GBExtracting  2.902GB/2.962GB
Status: Downloaded newer image for fastdotai/fastai:latest
 ---> 5e12d2f6fee1
Step 2/8 : LABEL maintainer="Raj Kadiyala"
 ---> Running in dd914707e7b5
Removing intermediate container dd914707e7b5
 ---> 4a6f33c

Add docker images architecture to explain what is going on
show the Dockerfile

In [3]:
!docker images

REPOSITORY         TAG       IMAGE ID       CREATED         SIZE
fastai             latest    23cced0fcdc8   8 seconds ago   8.25GB
fastdotai/fastai   latest    5e12d2f6fee1   9 hours ago     8.22GB


## Set the ecr details and tags 
Lets set a few params here like ecr name space , tag name etc.

In [4]:
from sagemaker import get_execution_role
import boto3
ecr_namespace = "sagemaker-training-containers/"
prefix = "script-mode-container-fastai"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

In [5]:
tag_name

'406078665760.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai:latest'

In [6]:
!docker tag fastai $tag_name

### ECR Repository and push steps

All of these can be scripted out but they are laid out this way for transparency and step evolution understanding

In [7]:
!$(aws ecr get-login --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [8]:
!aws ecr create-repository --repository-name $ecr_repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:406078665760:repository/sagemaker-training-containers/script-mode-container-fastai",
        "registryId": "406078665760",
        "repositoryName": "sagemaker-training-containers/script-mode-container-fastai",
        "repositoryUri": "406078665760.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai",
        "createdAt": 1637076037.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [9]:
!docker push $tag_name

The push refers to repository [406078665760.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai]

6fa1c0ed: Preparing 
bccc2365: Preparing 
27f41bbe: Preparing 
937049be: Preparing 
9e58289c: Preparing 
35851702: Preparing 
8ee652fb: Preparing 
6e590844: Preparing 
bf18a086: Preparing 
2f7b7495: Preparing 
a5c5c23f: Preparing 
a0a7e853: Preparing 
4fc1cff8: Preparing 
8b8f01e0: Preparing 
1b69538d: Preparing 
fcbe6a14: Preparing 
0881ce19: Preparing 
fcbe6a14: Pushed   6.197GB/6.162GBPushing  5.025MB/28.34MBPushing    487MB/6.162GBPushing  2.673GB/6.162GBPushing  3.135GB/6.162GBPushing  3.335GB/6.162GBPushing   3.54GB/6.162GBPushing  4.708GB/6.162GBPushing  4.855GB/6.162GBPushing  4.866GB/6.162GBPushing  5.104GB/6.162GBPushing  5.116GB/6.162GBPushing  5.154GB/6.162GBPushing  5.966GB/6.162GBPushing   6.05GB/6.162GBlatest: digest: sha256:43ae68fc1ffdf3ff9cd8e9297f9506de27fbe676c3f138a3ce51572438a1da35 size: 4295


In [10]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format( account_id, region, ecr_repository_name)
print(container_image_uri)

406078665760.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai:latest


In [12]:
!aws s3 ls s3://rkadiy-data-bucket/train


An error occurred (NoSuchBucket) when calling the ListObjectsV2 operation: The specified bucket does not exist


#### Call your custom container to train the model

In [11]:
import sagemaker
import json

# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}


hyperparameters = json_encode_hyperparameters({"lr":1e-03})

est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    #train_instance_type="local",  # we use local mode
    instance_type='ml.m5.4xlarge',
    base_job_name=prefix,
    hyperparameters=hyperparameters,
)

train_config = sagemaker.session.TrainingInput('s3://rkadiy-data-bucket/train')

est.fit({"train": train_config})

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://rkadiy-data-bucket/train" given in input data source. Please ensure that the bucket exists in the selected region (us-west-2), that objects exist under that S3 prefix, and that the role "arn:aws:iam::406078665760:role/SageMakerRepoRole" has "s3:ListBucket" permissions on bucket "rkadiy-data-bucket". Error message from S3: The specified bucket does not exist